In [1]:
# Load all dependencies
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [2]:
# Load CIFar-10 dataset
BATCH_SIZE=32
train_dataset = datasets.CIFAR10(root='./data',
                                             train=True,
                                             download=True,
                                             transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data',
                                            train=False,
                                            download=True,
                                            transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=False)

print(train_loader.dataset)


100.0%


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1=nn.Conv2d(
            in_channels=3,
            out_channels=8,
            kernel_size=3,
            padding=1)
        self.conv2=nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            padding=1)
        self.pool=nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1=nn.Linear(8*8*16,64)
        self.fc2=nn.Linear(64,32)
        self.fc3=nn.Linear(32,10)

    def forward(self, x):
        x=self.conv1(x)
        x=torch.relu(x)
        x=self.pool(x)
        x=self.conv2(x)
        x=torch.relu(x)
        x=self.pool(x)

        x=x.view(-1, 8*8*16)
        x=self.fc1(x)
        x=torch.relu(x)
        x=self.fc2(x)
        x=torch.relu(x)
        x=self.fc3(x)
        x=torch.log_softmax(x, dim=1)
        return x

In [4]:
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using PyTorch version: {torch.__version__}, Device: {DEVICE}")

model=CNN().to(DEVICE)
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
criterion=nn.CrossEntropyLoss()

Using PyTorch version: 2.7.0+cu126, Device: cpu


In [5]:
def train(model, train_loader,optimizer, log_interval):
    model.train()
    for batch_idx,(image,label) in enumerate(train_loader):
        image=image.to(DEVICE)
        label=label.to(DEVICE)
        optimizer.zero_grad()
        output=model(image)
        loss=criterion(output,label)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [6]:
def evaluate(model,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for image,label in test_loader:
            image=image.to(DEVICE)
            label=label.to(DEVICE)
            output=model(image)
            test_loss+=criterion(output,label).item()
            pred=output.argmax(dim=1, keepdim=True)
            correct+=pred.eq(label.view_as(pred)).sum().item()

    test_loss/=len(test_loader.dataset)
    test_accuracy=100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
EPOCHS=10
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval=200)
    test_loss,test_accuracy=evaluate(model, test_loader)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%\n')


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.070481
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.957054
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.939635
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.849104
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.960079
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.783564
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.936694
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.289720

Test set: Average loss: 0.0327, Accuracy: 63.07%

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.837698
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.964133
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.017292
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.161956
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.980478
Train Epoch: 2 [32000/50000 (64%)]	Loss: 0.883963
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.726088
Train Epoch: 2 [44800/50000 (90%)]	Loss: 0.860680

Test set: Average loss: 0.0327, Accuracy: 63.62%

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.303686
Train Epoch: 3 [6400/50000 (13%)]	Loss: 0.843108
Train Epoch: 3 [